References

https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameNaFunctions<br>

In [2]:
from pyspark.sql import *

--------------------------------------------------------------------------- 
 ImportError Traceback (most recent call last)
 <command-2750624936771074> in <module> () 
 1 from pyspark . sql import * 
 ----> 2 from pyspark . sql . Column import * 

 ImportError : No module named 'pyspark.sql.Column'

#### Get a dataframe for notebook tasks

In [4]:
%run ./adb_3_ingest_to_df

#### Cleanup nulls, bad values

In [6]:
# How many nulls do we have in a column

df_explicit.where(df_explicit["name"].isNull()).count()

Out[ 19 ]: 0

In [7]:
# How many non-nulls do we have in a column - alternate syntax

df_explicit.filter("name is not NULL").count()

Out[ 20 ]: 365

In [8]:
# Drop rows where ALL columns contain null

df1 = df_explicit.na.drop(how='all')
df1.count()

Out[ 21 ]: 365

In [9]:
# Drop rows where ANY column contains null
df2 = df1.na.drop(how='any')
df2.count()

Out[ 22 ]: 365

In [10]:
# Fill nulls in an int column with value 0

df3 = df2.na.fill(0, ["id"])

In [11]:
# Replace some values
df4 = df3\
  .na.replace("Palau", "PW", ["state"])\
  .na.replace("Federated States of Micronesia", "FSM", ["state"])

df4.count()

Out[ 24 ]: 365

In [12]:
group_by_column = "state"
count_column = "n"

# Group by state so we can count airports per state; rename count column (confusing with describe() otherwise); sort by count descending
# We also filter to US states only so we can use the Databricks map visualization
df4_usstates_only = df4\
  .filter("state NOT IN ('DC', 'FSM', 'PR', 'PW', 'TT', 'VI')")\
  .groupBy(group_by_column)\
  .count()\
  .withColumnRenamed("count", count_column)\
  .sort(group_by_column, ascending=True)

In [13]:
display(df4_usstates_only)

state,n
AK,22
AL,5
AR,4
AZ,5
CA,32
CO,11
CT,2
DE,1
FL,20
GA,8
